In [ ]:
import os
import json
from collections import defaultdict
from functools import partial
from typing import Dict, List

import random
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils import data
from trajdata import UnifiedDataset, SceneBatch, AgentBatch
from trajdata.data_structures.agent import AgentType
from trajdata.augmentation import NoiseHistories

from data_generation import (
    # custom batch
    SceneAgentBatch,
    SceneProcessor,
    # custom data
    custom_world_from_agent_tf,
    get_neigh_idxs,
    custom_agent_neigh_hist,
    custom_agent_fut,
    custom_collate_fn,
    transform_coords_np,
    # utils
    load_model
)

/home/abbas/miniconda3/envs/fsp/lib/python3.9/site-packages/wandb/sdk/internal/internal_api.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


In [ ]:
log_dir = '../../data/trained_models/trajectory_prediction'
model_dir = os.path.join(log_dir, "eth_loo-29_May_2025_22_55_51")

with open(os.path.join(model_dir, 'config.json'), 'r', encoding="utf-8") as config_json:
    hyperparams = json.load(config_json)
# device
hyperparams["device"] = "cpu"
hyperparams["trajdata_cache_dir"] = "../../data/pedestrian_datasets/.unified_data_cache"

desired_data=[
    "eupeds_eth-train",
]
max_agent_num = 20
data_dirs = {
    "eupeds_eth": "../../data/pedestrian_datasets/eth_ucy_peds",
}

In [3]:
attention_radius = defaultdict(
    lambda: 20.0
)  # Default range is 20m unless otherwise specified.
# attention_radius[(AgentType.PEDESTRIAN, AgentType.PEDESTRIAN)] = 5.0
interaction_radius = 5.0

history_sec = (0.1, hyperparams["history_sec"])
future_sec = (0.1, hyperparams["prediction_sec"])

input_noise = 0.0
augmentations = list()
if input_noise > 0.0:
    augmentations.append(NoiseHistories(stddev=input_noise))

num_scenes = 1

dataset = UnifiedDataset(
    desired_data=desired_data,
    centric="scene",
    # centric="agent",
    history_sec=history_sec,
    future_sec=future_sec,
    agent_interaction_distances=attention_radius,
    max_agent_num=max_agent_num,
    incl_robot_future=hyperparams["incl_robot_node"],
    incl_raster_map=hyperparams["map_encoding"],
    only_predict=[AgentType.PEDESTRIAN],
    no_types=[AgentType.UNKNOWN],
    augmentations=augmentations if len(augmentations) > 0 else None,
    standardize_data=False,
    num_workers=hyperparams["preprocess_workers"],
    cache_location=hyperparams["trajdata_cache_dir"],
    data_dirs=data_dirs,
    verbose=True,
    extras={
        "world_from_agent_tf": custom_world_from_agent_tf,
        "is_neigh": partial(get_neigh_idxs, interaction_radius=interaction_radius),
        "agent_neigh_hist_st": partial(custom_agent_neigh_hist, history_sec=history_sec),
        "agent_fut_st": partial(custom_agent_fut, future_sec=future_sec),
    }
)

print(f"# Data Samples: {len(dataset)}")

base_collate = dataset.get_collate_fn(pad_format="right")

dataloader = data.DataLoader(
    dataset,
    # collate_fn=dataset.get_collate_fn(pad_format="right"),
    collate_fn=partial(
        custom_collate_fn,
        history_sec=history_sec,
        future_sec=future_sec,
        base_collate=base_collate
        ),
    pin_memory=False if hyperparams["device"] == "cpu" else True,
    batch_size=num_scenes,
    shuffle=True,
    num_workers=hyperparams["preprocess_workers"],
    sampler=None,
)

Loading data for matched scene tags: ['train_loo-eupeds_eth']


Calculating Agent Data (Serially): 100%|██████████| 7/7 [00:00<00:00, 17538.91it/s]

7 scenes in the scene index.



Structuring Scene Data Index: 100%|██████████| 7/7 [00:00<00:00, 264505.66it/s]

# Data Samples: 4356


In [10]:
# batch: AgentBatch = next(iter(dataloader))
batch: SceneBatch = next(iter(dataloader))
scene_batch = SceneAgentBatch(batch)

In [ ]:
scene_processor = SceneProcessor(scene_batch) # ensure only one scene is passed

epoch = 15
predictor = load_model(model_dir=model_dir, epoch=epoch, hyperparams=hyperparams)
ego_positions = scene_processor.get_ego_positions(samples_per_scene=5)

In [ ]:
supports = scene_processor.scene_supports(ego_positions, risk=0.05, confidence=0.01)
supports

array([4, 3, 2, 3, 4])

In [ ]:
# generate ego position and support data
for batch in dataloader:
    scene_batch = SceneAgentBatch(batch)
    ego_positions = scene_processor.get_ego_positions(samples_per_scene=5)
    supports = scene_processor.scene_supports(ego_positions, risk=0.05, confidence=0.01)

In [ ]:
## Visualization
import random
import matplotlib.pyplot as plt

## initiate plot
xLim = 15
yLim = 15
fig, ax = plt.subplots()
ax.set_xlim(-xLim, xLim)
ax.set_ylim(-yLim, yLim)

ts = 0

for idx, human_pos in enumerate(flat_preds[ts]):
     # Create a circle for each agent
    color = (random.random(), random.random(), random.random()) # Generate a random color for each agent

    # Draw the agent's current position
    circle = plt.Circle(
        (human_pos[0], human_pos[1]),
        0.5,
        facecolor=color,
        edgecolor="k",
        lw=0.5,
        zorder=3,
    )
    ax.add_artist(circle)
    # ax.annotate(
    #     obs.agent_name[idx],
    #     (human_pos[0], human_pos[1]),
    #     fontsize=8,
    #     ha="center",
    #     va="center",
    # )

    # Plot future positions with decreasing transparency
    # alpha = 1.0  # Initial transparency
    # human_fut_positions_st = obs.agent_fut[idx, :, :2].cpu().numpy()
    # human_fut_positions = human_fut_positions_st @ obs.agents_from_world_tf[idx, :2, :2].cpu().numpy() + human_pos.reshape(1, 2)
    # for future_pos in human_fut_positions:
    #     alpha = 0.8*alpha
    #     if future_pos[0] < -xLim or future_pos[0] > xLim or future_pos[1] < -yLim or future_pos[1] > yLim or np.isnan(future_pos).any():
    #         continue
    #     # print("Future Pos:", future_pos)
    #     future_circle = plt.Circle(
    #         (future_pos[0], future_pos[1]),
    #         0.5,
    #         facecolor=color,
    #         edgecolor="k",
    #         lw=0.5,
    #         alpha=alpha,
    #         zorder=3,
    #     )
    #     ax.add_artist(future_circle)

for coeff in pred_coeffs[ts]:
    # Draw the constraint line
    a, b, c = coeff
    x_vals = np.linspace(-xLim, xLim, 100)
    y_vals = (-a * x_vals - c) / b if b != 0 else np.full_like(x_vals, np.nan)  # Handle vertical lines
    ax.plot(x_vals, y_vals, color='red', linestyle='--', alpha=0.5, lw=1)

robot_color = (1.0, 1.0, 0.0)  # Bright yellow color
robot_plot = plt.Circle(
            (robot_pos[0], robot_pos[1]),
            0.5,
            facecolor=robot_color,
            edgecolor="k",
            lw=0.5,
            zorder=3,
        )
ax.add_artist(robot_plot);

In [11]:
B = len(dataloader)
batch_maxes = np.empty((B,), dtype=float)

for i, batch in enumerate(dataloader):
    scene_batch = SceneAgentBatch(batch)
    lens = scene_batch.agent_hist_len
    batch_idx = torch.arange(lens.size(0), device=lens.device)
    time_idx  = lens - 1

    pos = scene_batch.agent_hist[batch_idx, time_idx, :2]
    pos_np = pos.detach().cpu().numpy()
    pos_np = pos_np.reshape(pos_np.shape[0], 1, 2)

    tf = scene_batch.world_from_agent_tf

    world_pos = transform_coords_np(pos_np, tf)

    batch_maxes[i] = np.nanmax(world_pos)

global_max = np.nanmax(batch_maxes)
global_max

15.613143692499998